In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import itertools
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
온라인 = pd.read_csv('/Users/yaelinjo/GitHub/Recosys_Project/온라인_전처리_final_32columns.csv')
온라인.head()

,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_0,action_type_1,...,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_PC,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,buy
0,1,1,NaN,1,12.0,11.0,0,1,1,0,...,0,0,0,1,0,1,0,0,1_1,0
1,1,1,NaN,2,23.0,14.0,0,2,1,0,...,0,0,0,1,0,1,0,0,1_1,0
2,1,1,NaN,3,37.0,14.0,0,3,1,0,...,0,0,0,1,0,1,0,0,1_1,0
3,1,2,NaN,1,42.0,15.0,1,1,1,0,...,0,0,0,1,0,1,0,0,1_2,0
4,1,2,NaN,2,57.0,14.0,1,2,1,0,...,0,0,0,1,0,1,0,0,1_2,0


In [3]:
온라인.columns[3:-2]

Index(['hit_seq', 'hit_pss_tm', 'time_length', 'holiday_diff', 'keyword',
       'action_type_0', 'action_type_1', 'action_type_2', 'action_type_3',
       'action_type_4', 'biz_unit_A01', 'biz_unit_A02', 'biz_unit_A03',
       'sess_dt_201907', 'sess_dt_201908', 'sess_dt_201909', 'trfc_src_DIRECT',
       'trfc_src_PORTAL_1', 'trfc_src_PORTAL_2', 'trfc_src_PORTAL_3',
       'trfc_src_PUSH', 'trfc_src_WEBSITE', 'trfc_src_unknown',
       'dvc_ctg_nm_PC', 'dvc_ctg_nm_mobile_app', 'dvc_ctg_nm_mobile_web',
       'dvc_ctg_nm_unknown'],
      dtype='object')

In [4]:
len(온라인.columns[3:-2])

27

# <br>
# 종속변수 생성
현재 세션의 구매 여부

In [5]:
# 현재 세션의 구매를 예측하기 위한 종속변수 생성
구매여부 = 온라인[['clnt_id', 'sess_id', 'buy']].groupby(['clnt_id', 'sess_id']).sum()
구매여부.buy = 구매여부.buy.apply(lambda x:0 if x == 0 else 1)
구매여부 = 구매여부.sort_index()
구매여부 = 구매여부.reset_index()

In [6]:
구매여부['unique_id'] = list(map(lambda x,y: str(x)+'_'+str(y), 구매여부.clnt_id, 구매여부.sess_id))

In [7]:
구매여부

,clnt_id,sess_id,buy,unique_id
0,1,1,0,1_1
1,1,2,0,1_2
2,2,1,1,2_1
3,2,2,0,2_2
4,2,3,0,2_3
...,...,...,...,...
367144,72428,2,0,72428_2
367145,72428,3,0,72428_3
367146,72428,4,0,72428_4
367147,72428,5,0,72428_5


# <br>
# 구매 이전의 클릭로그 대상

In [8]:
idx1 = 온라인.unique_id.drop_duplicates().index.tolist()
idx2 = idx1[1:] + [len(온라인)]
idx3 = []
for i, j in tqdm_notebook(zip(idx1, idx2), total=len(idx1)):
    temp = 온라인.buy.iloc[i:j]
    try:
        idx3.append(temp[temp == 1].index[0])
    except:
        idx3.append(j)

In [9]:
온라인_구매이전_idx = []

for i, j in tqdm_notebook(zip(idx1, idx3), total=len(idx1)):
    온라인_구매이전_idx.extend(온라인.iloc[i:j].index.tolist())

In [10]:
온라인2 = 온라인.iloc[온라인_구매이전_idx].copy()

In [11]:
온라인2.head()

,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_0,action_type_1,...,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_PC,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,buy
0,1,1,NaN,1,12.0,11.0,0,1,1,0,...,0,0,0,1,0,1,0,0,1_1,0
1,1,1,NaN,2,23.0,14.0,0,2,1,0,...,0,0,0,1,0,1,0,0,1_1,0
2,1,1,NaN,3,37.0,14.0,0,3,1,0,...,0,0,0,1,0,1,0,0,1_1,0
3,1,2,NaN,1,42.0,15.0,1,1,1,0,...,0,0,0,1,0,1,0,0,1_2,0
4,1,2,NaN,2,57.0,14.0,1,2,1,0,...,0,0,0,1,0,1,0,0,1_2,0


# <br>
# flatten

In [12]:
def to_flat(df):
    cc = df.groupby(['unique_id']).cumcount() + 1
    flat_df = df.set_index(['unique_id', cc]).unstack().sort_index(1, level=1)
    flat_df.columns = ['_'.join(map(str,i)) for i in flat_df.columns]
    flat_df.reset_index(inplace=True)
    return flat_df

In [13]:
온라인2.drop(['clnt_id', 'sess_id', 'trans_id', 'buy'], axis=1, inplace=True)

In [14]:
온라인_flat = to_flat(온라인2)

In [15]:
온라인_flat = 온라인_flat.merge(구매여부, left_on='unique_id', right_on='unique_id')
온라인_flat.sort_values(by=['clnt_id','sess_id'], inplace=True)
온라인_flat.reset_index(drop=True, inplace=True)

In [16]:
# max 357 * 27 features + 4 = 9643
온라인_flat.shape

(359779, 9643)

In [17]:
# 패딩처리 시에도 0으로 패딩했으니까
# NaN 값들은 0으로 대체
온라인_flat.fillna(0, inplace=True)

In [18]:
온라인_flat.head()

,unique_id,action_type_0_1,action_type_1_1,action_type_2_1,action_type_3_1,action_type_4_1,biz_unit_A01_1,biz_unit_A02_1,biz_unit_A03_1,dvc_ctg_nm_PC_1,...,trfc_src_DIRECT_357,trfc_src_PORTAL_1_357,trfc_src_PORTAL_2_357,trfc_src_PORTAL_3_357,trfc_src_PUSH_357,trfc_src_WEBSITE_357,trfc_src_unknown_357,clnt_id,sess_id,buy
0,1_1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0
1,1_2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,0
2,2_1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,1
3,2_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,0
4,2_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3,0


# <br>
# [2-2]현재 세션(1개)의 첫번째 구매 이전 클릭 로그 대상

In [19]:
from keras.preprocessing import sequence
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [20]:
features = 온라인_flat.columns[1:-3]

In [21]:
온라인_x = 온라인_flat.iloc[:, 1:-3]
온라인_x = np.array(온라인_x)

In [22]:
# session 당 구매 여부
온라인_y = 온라인_flat.buy

In [23]:
def make_padding_and_oversample(X, Y, length=10):
    smote = SMOTE(random_state=0)
    X_resampled, Y_resampled = smote.fit_resample(X, Y)
    return X_resampled, Y_resampled

In [ ]:
X_resampled, Y_resampled = make_padding_and_oversample(온라인_x, 온라인_y, length=10)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, Y_resampled, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

# <br>
# 모델링

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import xgboost as xgb

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.layers import Dropout

import pickle
from joblib import dump, load

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
scores = dict()

### Gaussian Naive Bayes

In [ ]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

In [ ]:
scores['Gaussian Naive Bayes'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred),
                                  precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [ ]:
# dump(clf, '1-3-Gaussian.joblib')

### Decision Tree

In [ ]:
clf2 = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
y_pred = clf2.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

In [ ]:
scores['Decision Tree'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred),
                           precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [ ]:
feature_check = {}
for name, importance in zip(features, clf2.feature_importances_):
    feature_check[name] = importance

In [ ]:
pd.DataFrame.from_dict(feature_check, orient='index', columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

In [ ]:
# dump(clf2, '1-3-DecisionTree.joblib')

### XGBOOST

In [ ]:
clf3 = xgb.XGBClassifier(learning_rate = 0.05, n_estimators=300, max_depth=3, verbosity=2, random_state=0).fit(X_train, y_train)
y_pred = clf3.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

In [ ]:
scores['XGboost'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [ ]:
feature_check = {}
for name, importance in zip(features, clf3.feature_importances_):
    feature_check[name] = importance

In [ ]:
pd.DataFrame.from_dict(feature_check, orient='index', columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

In [ ]:
# dump(clf3, '1-3-xgb.joblib')

### Logistic Regression

In [ ]:
clf4 = LogisticRegression(max_iter=5000, random_state=0).fit(X_train, y_train)
y_pred = clf4.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

In [ ]:
scores['Logistic Regression'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [ ]:
# dump(clf4, '1-3-LogisticRegression.joblib')

### Linear SVM

In [ ]:
clf5 = LinearSVC(random_state=0).fit(X_train, y_train)
y_pred = clf5.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

In [ ]:
scores['Linear SVM'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [ ]:
# dump(clf4, '1-3-LinearSVM.joblib')

### MLP(DNN)

In [ ]:
def models(train):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(train.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr= 0.001, rho = 0.9), metrics=['acc', f1_m, precision_m, recall_m])
    return model

In [ ]:
model = models(X_train)
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=25, batch_size=1000, validation_data=(X_test, y_test), verbose=2, shuffle=True)

In [ ]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)

In [ ]:
scores['DNN'] = [accuracy, f1_score, precision, recall]

In [ ]:
print(loss)
print(accuracy)
print(f1_score)
print(precision)
print(recall)

In [ ]:
# model.save('1-3-DNN.h5')

In [ ]:
pd.DataFrame.from_dict(scores, orient='index', columns=['Accuracy', 'F1-Score', 'Precision', 'Recall'])